<a href="https://colab.research.google.com/github/antonpetkoff/identifier-suggestion/blob/master/notebooks/00-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import utilities
import os
import shutil
from subprocess import check_output

In [2]:
!nvidia-smi

Wed Jul  1 15:58:36 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%env WORKSPACE_DIR=/content/gdrive/My Drive/src

# TODO: how can one read an environment variable?!?
%cd '/content/gdrive/My Drive/src'

env: WORKSPACE_DIR=/content/gdrive/My Drive/src
/content/gdrive/My Drive/src


In [5]:
timestamp = check_output(['date', '-u', "+%Y-%m-%dT%H-%M-%S"]).decode('utf-8').strip()

os.environ['PROJECT_DIR'] = os.path.join(
    os.environ['WORKSPACE_DIR'],
    f'identifier-suggestion-{timestamp}',
)

In [6]:
!git clone https://github.com/antonpetkoff/identifier-suggestion.git --depth 1 "${PROJECT_DIR}"

Cloning into '/content/gdrive/My Drive/src/identifier-suggestion-2020-07-01T15-59-13'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 114 (delta 1), reused 67 (delta 1), pack-reused 0
Receiving objects: 100% (114/114), 984.71 KiB | 11.45 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [7]:
os.chdir(os.environ['PROJECT_DIR'])

In [8]:
!pwd
!ls -l

/content/gdrive/My Drive/src/identifier-suggestion-2020-07-01T15-59-13
total 69
drwx------  4 root root  4096 Jul  1 15:59 data
-rw-------  1 root root 35149 Jul  1 15:59 LICENSE
drwx------  2 root root  4096 Jul  1 15:59 notebooks
-rw-------  1 root root 10478 Jul  1 15:59 README.md
drwx------  3 root root  4096 Jul  1 15:59 reports
drwx------  2 root root  4096 Jul  1 15:59 requirements
drwx------ 13 root root  4096 Jul  1 15:59 src
drwx------  3 root root  4096 Jul  1 15:59 vscode-extension


In [9]:
%tensorflow_version 2.x

import tensorflow
print(tensorflow.__version__)

2.2.0


In [10]:
# Google Colab has standard libraries like numpy, pandas, matplotlib and TF (of course) pre-installed
!pip install -r requirements/colab.txt

     |████████████████████████████████| 92kB 3.7MB/s 
     |████████████████████████████████| 1.4MB 19.8MB/s 
     |████████████████████████████████| 4.3MB 32.3MB/s 
     |████████████████████████████████| 112kB 50.5MB/s 
     |████████████████████████████████| 460kB 51.8MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=6bff850157c2af465a8764b152cdea01fa918733995f57a3d5ac77cbb46f6eba
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=c1dc07ca37020105d0f903158cdb26553739ac82dca275d22854dd6c27219252
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f

In [11]:
# provide secrets to the project, e.g. access to wandb
shutil.copy(
    os.path.join(os.environ['WORKSPACE_DIR'], 'secrets/.env'),
    os.environ['PROJECT_DIR']
)

'/content/gdrive/My Drive/src/identifier-suggestion-2020-07-01T15-59-13/.env'

In [12]:
# monkey-patch (mock) os.symlink to be a noop, because wandb.save() uses it, but it is not supported by Google Colab Notebooks
os.symlink = lambda *x: print('Executing mocked noop symlink with arguments', x)

In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [14]:
from argparse import Namespace
from src.pipelines.baseline import run

params = {
  'dir_data': '../data/processed/subtoken/', # get the input data from the shared directory
  'file_checkpoint_dir': 'models/checkpoints/baseline/',
  'dir_preprocessed_data': 'data/processed/seq2seq/',
  
  'experiment_name': 'random_search_0_0',
  'max_input_length': 120,
  'max_output_length': 6,
  'input_vocab_size': 5800,
  'output_vocab_size': 5800,
  'input_embedding_dim': 32,
  'output_embedding_dim': 32,
  'latent_dim': 640,
  'learning_rate': 0.085805,
  'dropout_rate': 0.118569,
  'batch_size': 512,

  'epochs': 200,
  'early_stopping_patience': 4,
  'early_stopping_min_delta': 0.0,
  'evaluation_dataset': 'validation',
  'random_seed': 1,
}

run(Namespace(**params))

Initializing logger


Experiment parameters:  Namespace(batch_size=1024, dir_data='../data/processed/subtoken/', dir_preprocessed_data='../data/processed/seq2seq/', dropout_rate=0.05, early_stopping_min_delta=0.001, early_stopping_patience=3, epochs=50, evaluation_dataset='validation', file_checkpoint_dir='models/checkpoints/baseline/', input_embedding_dim=50, input_vocab_size=5000, latent_dim=320, learning_rate=0.001, max_input_length=128, max_output_length=8, output_embedding_dim=50, output_vocab_size=5000, random_seed=1)
Loading preprocessed files...
Loaded input vocabulary.
Loaded output vocabulary.
Loaded preprocessed files.
Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
EncoderEmbedding (Embedding) multiple                  250000    
_________________________________________________________________
EncoderLSTM (LSTM)           multiple                  474880    
Total params: 724,880
Trainable param

wandb: Visualizing heatmap.


epoch 1 evaluation on test data time: 19.938061714172363 sec
epoch evaluation:  {'epoch': 1, 'epoch_test_loss': <tf.Tensor: shape=(), dtype=float32, numpy=2.7829003>, 'test_rouge_1_p': 0.19762961647727273, 'test_rouge_1_r': 0.08504144200293753, 'test_rouge_1_f1': 0.11387190565229344, 'test_rouge_2_p': 0.0014965503246753247, 'test_rouge_2_r': 0.0007055769751082251, 'test_rouge_2_f1': 0.0008794799880179346, 'test_rouge_3_p': 0.0, 'test_rouge_3_r': 0.0, 'test_rouge_3_f1': 0.0, 'test_rouge_L_p': 0.19760213744588745, 'test_rouge_L_r': 0.08503165825989485, 'test_rouge_L_f1': 0.11385787578335285}


wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


...

wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.
wandb: Visualizing heatmap.


epoch 6 - batch 10 - loss 2.3728880882263184
epoch 6 - batch 20 - loss 2.4034371376037598
epoch 6 - batch 30 - loss 2.282356023788452
epoch 6 - batch 40 - loss 2.2355151176452637
epoch 6 - batch 50 - loss 2.3599236011505127
epoch 6 - batch 60 - loss 2.321645736694336
epoch 6 - batch 70 - loss 2.3520872592926025
epoch 6 - batch 80 - loss 2.2762866020202637
epoch 6 - batch 90 - loss 2.3052399158477783
epoch 6 - batch 100 - loss 2.3899552822113037
epoch 6 - batch 110 - loss 2.281309127807617
epoch 6 - batch 120 - loss 2.210811138153076
epoch 6 - batch 130 - loss 2.2766451835632324
epoch 6 - batch 140 - loss 2.300865650177002
epoch 6 - batch 150 - loss 2.3283493518829346
epoch 6 - batch 160 - loss 2.2724215984344482
epoch 6 - batch 170 - loss 2.3418350219726562
epoch 6 - batch 180 - loss 2.2927050590515137
epoch 6 - batch 190 - loss 2.278414726257324
epoch 6 - batch 200 - loss 2.1781952381134033
epoch 6 - batch 210 - loss 2.337951898574829
epoch 6 - batch 220 - loss 2.2676749229431152
epoc

KeyboardInterrupt: ignored

Error in callback <function _init_jupyter.<locals>.cleanup at 0x7fedf9c36488> (for post_run_cell):


KeyboardInterrupt: ignored